In [26]:
# https://github.com/devopshq/artifactory
from artifactory import ArtifactoryPath
import os
base_path = "http://127.0.0.1:8086/artifactory"

In [27]:
# Directory Walking
def directory_walk(path):
    
    path = ArtifactoryPath(path)
    for p in path:
        print(p)

In [28]:
# User and password
path = ArtifactoryPath("http://127.0.0.1:8086/artifactory/snet_test", auth=("admin", "password"))

for p in path:
    print(p)

http://127.0.0.1:8086/artifactory/snet_test/text_artifact
http://127.0.0.1:8086/artifactory/snet_test/tools


In [44]:
with open('inputs/text_artifact-1.0.txt', 'w') as f:
    f.write('sample data')
with open('inputs/text_artifact-1.1.txt', 'w') as f:
    f.write('sample data')
with open('inputs/text_artifact-1.2.txt', 'w') as f:
    f.write('sample data')


In [45]:
path = ArtifactoryPath("http://127.0.0.1:8086/artifactory/snet_test/text_artifact", auth=("admin", "password"))

if not path.exists():
    path.mkdir()

In [46]:
# uploading based on simple-layout ([orgPath]/[module]/[module]-[baseRev].[ext])
repo_path = "/snet_test"
org_path = "/tools"
module = "/text_artifact"

repo = base_path+repo_path+org_path+module

path = ArtifactoryPath(repo, auth=("admin", "password"))

if not path.exists():
    path.mkdir()

path.deploy_file("./inputs/text_artifact-1.0.txt")
path.deploy_file("./inputs/text_artifact-1.1.txt")
path.deploy_file("./inputs/text_artifact-1.2.txt")

In [32]:
# retrieve artifact hashes
directory_walk("http://127.0.0.1:8086/artifactory/snet_test/tools/text_artifact")

http://127.0.0.1:8086/artifactory/snet_test/tools/text_artifact/text_artifact-1.0.txt
http://127.0.0.1:8086/artifactory/snet_test/tools/text_artifact/text_artifact-1.1.txt
http://127.0.0.1:8086/artifactory/snet_test/tools/text_artifact/text_artifact-1.2.txt


In [33]:
# retrieve md5 sum and other stats

path = ArtifactoryPath("http://127.0.0.1:8086/artifactory/snet_test/tools/text_artifact/text_artifact-1.0.txt")

# Get FileStat
stat = ArtifactoryPath.stat(path)
print(stat)
print(stat.md5)
print(stat.sha1)
print(stat.sha256)
print(stat.ctime)
print(stat.is_dir)
print(stat.size)

ArtifactoryFileStat(ctime=datetime.datetime(2019, 11, 25, 0, 52, 8, 347000, tzinfo=tzutc()), mtime=datetime.datetime(2019, 11, 25, 5, 43, 9, 3000, tzinfo=tzutc()), created_by='admin', modified_by='admin', mime_type='text/plain', size=11, sha1='3579307d55e123bde331b3eefce08090bea3fbe7', sha256='f107aac59dff1d49ebfedb7f03877eaa0297f9a7d3cff26edfc75406f222256d', md5='26e078b87fdaa3206ab8bf63a6096c07', is_dir=False, children=None)
26e078b87fdaa3206ab8bf63a6096c07
3579307d55e123bde331b3eefce08090bea3fbe7
f107aac59dff1d49ebfedb7f03877eaa0297f9a7d3cff26edfc75406f222256d
2019-11-25 00:52:08.347000+00:00
False
11


In [42]:
def download_direct(artifact, dest):      
    arti_path = ArtifactoryPath(artifact)
    
    with arti_path.open() as fd:
        with open(os.path.join(dest, arti_path.stem), "wb") as out:
            out.write(fd.read())

def download_artifact(artifact, dest):
    artifact_path = '/'.join([base_path, artifact['repo'], artifact['path'], artifact['name']])
       
    arti_path = ArtifactoryPath(artifact_path)
    
    with arti_path.open() as fd:
        with open(os.path.join(dest, artifact['name']), "wb") as out:
            out.write(fd.read())

In [35]:
# Retrieve latest artifact using aql (probably not exposed without credentials)
base_path = "http://127.0.0.1:8086/artifactory"
repo = "snet_test"
org = "tools"
module = "text_artifact"

aql = ArtifactoryPath(base_path, auth=("admin", "password"))

args = ["items.find", {"$and": [
        { "repo": {"$eq": repo} },
        {
            "$or": [
                {"path": {"$match": org+'/'+module}}
            ]
        }
        ]
},
".sort", {"$desc" : ["created"]},
".limit", [1]
]
artifacts_list = aql.aql(*args)

print(artifacts_list)

if artifacts_list:
    download_artifact(artifacts_list[0], 'downloads')

[{'repo': 'snet_test', 'path': 'tools/text_artifact', 'name': 'text_artifact-1.2.txt', 'type': 'file', 'size': 11, 'created': '2019-11-25T00:52:08.399Z', 'created_by': 'admin', 'modified': '2019-11-25T05:43:09.109Z', 'modified_by': 'admin', 'updated': '2019-11-25T05:43:09.120Z'}]


In [47]:
# Retrieve latest artifact using pathlib flavour
base_path = "http://127.0.0.1:8086/artifactory"
repo = "snet_test"
org = "tools"
module = "text_artifact"

path = ArtifactoryPath("http://127.0.0.1:8086/artifactory/snet_test/tools/text_artifact/")

newest_file = (max(path))
print(newest_file)

if newest_file:
    download_direct(newest_file, 'downloads')

http://127.0.0.1:8086/artifactory/snet_test/tools/text_artifact/text_artifact-1.2.txt
